In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, concat, lit,count, mean, sum as spark_sum
import warnings
warnings.filterwarnings('ignore')

In [2]:
spark = SparkSession.builder \
    .appName("Read CSV Files with PySpark") \
    .master("local[*]") \
    .getOrCreate()

os_base = "file:///root/lol/"


2023-07-13 21:48:10,147 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2023-07-13 21:48:11,591 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


| Field Name | Description              |
|------|----------|
| name | Name of the hero    |
| id   | Id of the hero |

In [3]:
champs = spark.read.csv(os_base + 'data/champs.csv', header=True, inferSchema=True)
champs.show(3)

+--------+---+
|    name| id|
+--------+---+
|     Jax| 24|
|    Sona| 37|
|Tristana| 18|
+--------+---+
only showing top 3 rows



| Field Name | Description              |
|------------|--------------------------|
| id         | Unique identifier of the match |
| gameid     | ID of the match         |
| platformid | ID of the platform where the match is located |
| queueid    | ID of the queue where the match is located |
| seasonid   | ID of the season where the match is located |
| duration   | Duration of the match (in seconds) |
| creation   | Timestamp of match creation |
| version    | Version of the game used in the match |
| position   | Participant's position in the match |

In [4]:
matches = spark.read.csv(os_base + 'data/matches.csv', header=True, inferSchema=True)
matches.show(3)

+---+----------+----------+-------+--------+--------+-------------+-------------+
| id|    gameid|platformid|queueid|seasonid|duration|     creation|      version|
+---+----------+----------+-------+--------+--------+-------------+-------------+
| 10|3187427022|      EUW1|    420|       8|    1909|1495068946860|7.10.187.9675|
| 11|3187425281|      EUW1|    420|       8|    1693|1495066760778|7.10.187.9675|
| 12|3187269801|      EUW1|    420|       8|    1482|1495053375889|7.10.187.9675|
+---+----------+----------+-------+--------+--------+-------------+-------------+
only showing top 3 rows



| Field Name | Description              |
|------------|--------------------------|
| id         | Unique identifier of the participant |
| matchid    | ID of the match participated by the participant |
| player     | ID of the participant |
| championid | ID of the hero used by the participant |
| ss1        | ID of the participant's Summoner Skill 1 |
| ss2        | ID of the participant's Summoner Skill 2 |
| role       | Participant's role in the match |
| position   | Participant's position in the match |

In [5]:
participants = spark.read.csv(os_base + 'data/participants.csv', header=True, inferSchema=True)
participants.show(3)

+---+-------+------+----------+---+---+-----------+--------+
| id|matchid|player|championid|ss1|ss2|       role|position|
+---+-------+------+----------+---+---+-----------+--------+
|  9|     10|     1|        19|  4| 11|       NONE|  JUNGLE|
| 10|     10|     2|       267|  3|  4|DUO_SUPPORT|     BOT|
| 11|     10|     3|       119|  7|  4|  DUO_CARRY|     BOT|
+---+-------+------+----------+---+---+-----------+--------+
only showing top 3 rows



| Field Name              | Description                                                  |
|-------------------------|--------------------------------------------------------------|
| id                      | Participant ID                                               |
| matchid                 | Match ID                                                     |
| player                  | Player number in the match                                   |
| championid              | ID of the used hero                                          |
| ss1                     | Summoner Skill 1                                             |
| ss2                     | Summoner Skill 2                                             |
| role                    | Player role (e.g., SOLO; DUO_SUPPORT; DUO_CARRY; NONE)       |
| position                | Player position (e.g., TOP; JUNGLE; MID; BOT)                |
| win                     | Match win status                                             |
| item1                   | Item 1                                                       |
| item2                   | Item 2                                                       |
| item3                   | Item 3                                                       |
| item4                   | Item 4                                                       |
| item5                   | Item 5                                                       |
| item6                   | Item 6                                                       |
| trinket                 | Item 7                                                       |
| kills                   | Number of kills                                              |
| deaths                  | Number of deaths                                             |
| assists                 | Number of assists                                            |
| largestkillingspree     | Largest killing spree                                        |
| largestmultikill        | Largest multi-kill                                           |
| killingsprees           | Number of killing sprees                                     |
| longesttimespentliving  | Longest time spent living                                    |
| doublekills             | Number of double kills                                       |
| triplekills             | Number of triple kills                                       |
| quadrakills             | Number of quadra kills                                       |
| pentakills              | Number of penta kills                                        |
| legendarykills          | Number of legendary level kills                              |
| totdmgdealt             | Total damage dealt                                           |
| magicdmgdealt           | Magic damage dealt                                           |
| physicaldmgdealt        | Physical damage dealt                                        |
| truedmgdealt            | True damage dealt                                            |
| largestcrit             | Largest critical strike                                      |
| totdmgtochamp           | Total damage dealt to champions                              |
| magicdmgtochamp         | Magic damage dealt to champions                              |
| physdmgtochamp          | Physical damage dealt to champions                           |
| truedmgtochamp          | True damage dealt to champions                               |
| totheal                 | Total healing done                                           |
| totunitshealed          | Total units healed                                           |
| dmgselfmit              | Total damage self-mitigated                                  |
| dmgtoobj                | Damage dealt to objectives                                   |
| dmgtoturrets            | Damage dealt to turrets                                      |
| visionscore             | Vision score                                                 |
| timecc                  | Total time controlling enemy champions                       |
| totdmgtaken             | Total damage taken                                           |
| magicdmgtaken           | Magic damage taken                                           |
| physdmgtaken            | Physical damage taken                                        |
| truedmgtaken            | True damage taken                                            |
| goldearned              | Total gold earned                                            |
| goldspent               | Total gold spent                                             |
| turretkills             | Number of turrets destroyed                                  |
| inhibkills              | Number of inhibitors destroyed                               |
| totminionskilled        | Total minions killed                                         |
| neutralminionskilled    | Number of neutral minions killed                             |
| ownjunglekills          | Number of jungle minions killed in player's own jungle       |
| enemyjunglekills        | Number of jungle minions killed in enemy's jungle            |
| totcctimedealt          | Total time of crowd control effects dealt to enemy champions (in milliseconds) |
| champlvl                | Level of player's champion                                   |
| pinksbought             | Number of pink vision wards bought                           |
| wardsbought             | Number of vision wards bought (includes control wards and pink wards) |
| wardsplaced             | Number of vision wards placed (includes control wards and pink wards) |
| wardskilled             | Number of enemy vision wards destroyed (includes control wards and pink wards) |
| firstblood              | First blood (0 = no, 1 = yes)                                |

In [6]:
stats1 = spark.read.csv(os_base + 'data/stats1.csv', header=True, inferSchema=True)
stats2 = spark.read.csv(os_base + 'data/stats2.csv', header=True, inferSchema=True)
stats = stats1.union(stats2)
stats.show(3)

2023-07-12 21:52:55,395 WARN util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---+---+-----+-----+-----+-----+-----+-----+-------+-----+------+-------+-------------------+----------------+-------------+----------------------+-----------+-----------+-----------+----------+--------------+-----------+-------------+----------------+------------+-----------+-------------+---------------+--------------+--------------+-------+--------------+----------+--------+------------+-----------+------+-----------+-------------+------------+------------+----------+---------+-----------+----------+----------------+--------------------+--------------+----------------+--------------+--------+-----------+-----------+-----------+-----------+----------+
| id|win|item1|item2|item3|item4|item5|item6|trinket|kills|deaths|assists|largestkillingspree|largestmultikill|killingsprees|longesttimespentliving|doublekills|triplekills|quadrakills|pentakills|legendarykills|totdmgdealt|magicdmgdealt|physicaldmgdealt|truedmgdealt|largestcrit|totdmgtochamp|magicdmgtochamp|physdmgtochamp|truedmgtochamp

#### Data Join 

In [7]:
df = participants.join(stats, on=[participants['id'] == stats['id']], how='left').drop(stats['id'])
df = df.join(champs, on=[df['championid'] == champs['id']], how='left').drop(champs['id'])
df = df.join(matches, on=[df['matchid'] == matches['id']], how='left').drop(matches['id'])

df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- matchid: integer (nullable = true)
 |-- player: integer (nullable = true)
 |-- championid: integer (nullable = true)
 |-- ss1: integer (nullable = true)
 |-- ss2: integer (nullable = true)
 |-- role: string (nullable = true)
 |-- position: string (nullable = true)
 |-- win: integer (nullable = true)
 |-- item1: integer (nullable = true)
 |-- item2: integer (nullable = true)
 |-- item3: integer (nullable = true)
 |-- item4: integer (nullable = true)
 |-- item5: integer (nullable = true)
 |-- item6: integer (nullable = true)
 |-- trinket: integer (nullable = true)
 |-- kills: integer (nullable = true)
 |-- deaths: integer (nullable = true)
 |-- assists: integer (nullable = true)
 |-- largestkillingspree: integer (nullable = true)
 |-- largestmultikill: integer (nullable = true)
 |-- killingsprees: integer (nullable = true)
 |-- longesttimespentliving: integer (nullable = true)
 |-- doublekills: integer (nullable = true)
 |-- triplekills: integ

#### Data preprocessing, clear NULL values, and attach default values

In [13]:
null_counts = df.select([spark_sum(col(c).isNull().cast("int")).alias(c) for c in df.columns])

null_counts.show()

+---+-------+------+----------+---+---+----+--------+---+-----+-----+-----+-----+-----+-----+-------+-----+------+-------+-------------------+----------------+-------------+----------------------+-----------+-----------+-----------+----------+--------------+-----------+-------------+----------------+------------+-----------+-------------+---------------+--------------+--------------+-------+--------------+----------+--------+------------+-----------+------+-----------+-------------+------------+------------+----------+---------+-----------+----------+----------------+--------------------+--------------+----------------+--------------+--------+-----------+-----------+-----------+-----------+----------+----+------+----------+-------+--------+--------+--------+-------+
| id|matchid|player|championid|ss1|ss2|role|position|win|item1|item2|item3|item4|item5|item6|trinket|kills|deaths|assists|largestkillingspree|largestmultikill|killingsprees|longesttimespentliving|doublekills|triplekills|qua

In [14]:

df = df.dropna()

df = df.withColumn('wardsbought', when(col('wardsbought') == '\\N', 0).otherwise(col('wardsbought')))
df = df.withColumn('wardsbought', col('wardsbought').cast('int'))

In [15]:

spark_df = df.withColumn(
    "adjposition",
    when(col("role").isin(["DUO_SUPPORT", "DUO_CARRY"]), col("role")).otherwise(col("position"))
)

spark_df = spark_df.withColumn("team", when(col("player") <= 5, "1").otherwise("2"))
spark_df = spark_df.withColumn("team_role", concat(col("team"), lit(" - "), col("adjposition")))

team_roles = [
    "1 - MID",
    "1 - TOP",
    "1 - DUO_SUPPORT",
    "1 - DUO_CARRY",
    "1 - JUNGLE",
    "2 - MID",
    "2 - TOP",
    "2 - DUO_SUPPORT",
    "2 - DUO_CARRY",
    "2 - JUNGLE",
]
remove_index = []

# Remove non-typical teams
df_remove = (
    spark_df.filter(col("team_role").isin(team_roles))
    .groupBy("matchid", "team_role")
    .agg(count("team_role").alias("team_role_count"))
    .filter(col("team_role_count") > 1)
    .select("matchid")
)

remove_index = [row["matchid"] for row in df_remove.collect()]

bot_matchids = (
    spark_df.filter(col("adjposition") == "BOT").select("matchid").distinct()
)

remove_index.extend([row["matchid"] for row in bot_matchids.collect()])
remove_index = list(set(remove_index))

spark_df = spark_df.filter(~col("matchid").isin(remove_index))

In [16]:
# Calculate KDA
spark_df = spark_df.withColumn(
    "KDA",
    when(col("deaths") == 0, (col("kills") + col("assists")) / 0.5)
    .otherwise((col("kills") + col("assists")) / col("deaths")),
)

drop_columns = [
    "position",
    "team",
    "team_role",
    "gameid",
    "queueid",
    "creation",
    "role",
    "wardsbought",
    "name",
    "platformid",
    "version",
    "adjposition",
    "item1",
    "item2",
    "item3",
    "item4",
    "item5",
    "item6",
    "ss1",
    "ss2",
]
fea_df = spark_df.drop(*drop_columns)



fea_df.coalesce(1).write.csv("file:///root/lol/spark_feature", mode="overwrite", header=True)

2023-07-12 22:07:46,158 WARN scheduler.DAGScheduler: Broadcasting large task binary with size 1045.4 KiB
2023-07-12 22:09:16,005 WARN scheduler.DAGScheduler: Broadcasting large task binary with size 2028.3 KiB


In [17]:
spark_df.coalesce(1).write.csv("file:///root/lol/total_clean_data", mode="overwrite", header=True)

2023-07-12 22:15:20,890 WARN scheduler.DAGScheduler: Broadcasting large task binary with size 1045.4 KiB
2023-07-12 22:15:33,103 WARN scheduler.DAGScheduler: Broadcasting large task binary with size 2036.2 KiB


In [8]:
# Group calculation of the number of wins, number of games, average number of kills, average number of deaths, and average number of assists for each hero
champs_stats = spark_df.groupBy('name').agg(
    spark_sum('win').alias('win'),
    count('*').alias('total matches'),
    mean('kills').alias('K'),
    mean('deaths').alias('D'),
    mean('assists').alias('A')
)

# Calculate the winning rate/probability and based on a 50% winning rate/probability
champs_stats = champs_stats.withColumn('win rate(%)', col('win') / col('total matches') * 100)
champs_stats = champs_stats.withColumn('win rate-50%', col('win rate(%)') - 50)

champs_stats.coalesce(1).write.csv("file:///root/lol/champs_stats", mode="overwrite", header=True)